In [1]:
import os
os.chdir('/nfs/homedirs/ayle/guided-research/SNIP-it/')

In [2]:
import torch
from torchvision import datasets, transforms
import foolbox as fb
from experiments.main import load_checkpoint
from models import GeneralModel
from models.statistics.Metrics import Metrics
from utils.config_utils import *
from utils.model_utils import *
from utils.system_utils import *
from utils.attacks_utils import get_attack
from torch.utils.data.dataset import Dataset
from copy import deepcopy
from utils.metrics import calculate_aupr, calculate_auroc
from utils.attacks_utils import construct_adversarial_examples
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.nn.functional as F
from torch.distributions import Categorical
from scipy.spatial.distance import cosine
from scipy.stats import ks_2samp

In [3]:
arguments = dict({
'eval_freq': 1000,  # evaluate every n batches
    'save_freq': 1e6,  # save model every n epochs, besides before and after training
    'batch_size': 512,  # size of batches, for Imagenette 128
    'seed': 1234,  # random seed
    'max_training_minutes': 6120 , # one hour and a 45 minutes max, process killed after n minutes (after finish of epoch)
    'plot_weights_freq': 50, # plot pictures to tensorboard every n epochs
    'prune_freq': 1, # if pruning during training: how long to wait before starting
    'prune_delay': 0, # "if pruning during training: 't' from algorithm box, interval between pruning events, default=0
    'prune_to': 0,
    'epochs': 0,
    'rewind_to': 0, # rewind to this epoch if rewinding is done
    'snip_steps': 5, # 's' in algorithm box, number of pruning steps for 'rule of thumb', TODO
    'snip_iter': 1000,
    'pruning_rate': 0.0, # pruning rate passed to criterion at pruning event. however, most override this
    'growing_rate': 0.0000 , # grow back so much every epoch (for future criterions)
    'pruning_limit': 0.0,  # Prune until here, if structured in nodes, if unstructured in weights. most criterions use this instead of the pruning_rate
    'local_pruning': 0,
    'learning_rate': 2e-3,
    'grad_clip': 10,
    'grad_noise': 0 , # added gaussian noise to gradients
    'l2_reg': 5e-5 , # weight decay
    'l1_reg': 0 , # l1-norm regularisation
    'lp_reg': 0 , # lp regularisation with p < 1
    'l0_reg': 1.0 , # l0 reg lambda hyperparam
    'hoyer_reg': 0.001 , # hoyer reg lambda hyperparam
    'beta_ema': 0.999 , # l0 reg beta ema hyperparam

    'loss': 'CrossEntropy',
    'optimizer': 'ADAM',
    'model': 'ResNet18',  # ResNet not supported with structured
    'data_set': 'CIFAR10',
    'ood_data_set': 'SVHN',
    'ood_data_set_prune': 'SVHN',
    'prune_criterion': 'SNAP',  # options: SNIP, SNIPit, SNIPitDuring, UnstructuredRandom, GRASP, HoyerSquare, IMP, // SNAPit, StructuredRandom, GateDecorators, EfficientConvNets, GroupHoyerSquare
    'train_scheme': 'DefaultTrainer' , # default: DefaultTrainer
    'attack': 'FGSM',
    'epsilon': 6,
    'eval_ood_data_sets': ['SVHN', 'CIFAR100'],
    'eval_attacks': ['FGSM'],
    'eval_epsilons': [8, 48],

    'device': 'cuda',
    'results_dir': "tmp",

    'checkpoint_name': None,
    'checkpoint_model': None,

    'disable_cuda_benchmark': 1 , # speedup (disable) vs reproducibility (leave it)
    'eval': 0,
    'disable_autoconfig': 0 , # for the brave
    'preload_all_data': 0 , # load all data into ram memory for speedups
    'tuning': 0 , # splits trainset into train and validationset, omits test set

    'get_hooks': 0,
    'track_weights': 0 , # "keep statistics on the weights through training
    'disable_masking': 1 , # disable the ability to prune unstructured
    'enable_rewinding': 0, # enable the ability to rewind to previous weights
    'outer_layer_pruning': 1, # allow to prune outer layers (unstructured) or not (structured)
    'first_layer_dense': 0,
    'random_shuffle_labels': 0  ,# run with random-label experiment from zhang et al
    'l0': 0,  # run with l0 criterion, might overwrite some other arguments
    'hoyer_square': 0, # "run in unstructured DeephoyerSquare criterion, might overwrite some other arguments
    'group_hoyer_square': 0 ,# run in unstructured Group-DeephoyerSquare criterion, might overwrite some other arguments

    'disable_histograms': 0,
    'disable_saliency': 0,
    'disable_confusion': 0,
    'disable_weightplot': 0,
    'disable_netplot': 0,
    'skip_first_plot': 0,
    'disable_activations': 0,
    
#     'input_dim': [1, 28, 28],
#       'output_dim': 10,
#       'hidden_dim': [512],
#       'N': 60000,
    
    'input_dim': [3, 32, 32],
    'output_dim': 10,
    'hidden_dim': [512],
    'N': 60000,
    'mean': (0.4914, 0.4822, 0.4465),
    'std': (0.2471, 0.2435, 0.2616)
})

In [4]:
DATASET_PATH = '/nfs/students/ayle/guided-research/gitignored/data'

In [5]:
metrics = Metrics()
out = metrics.log_line
metrics._batch_size = arguments['batch_size']
metrics._eval_freq = arguments['eval_freq']
set_results_dir(arguments["results_dir"])

In [6]:
model: GeneralModel = find_right_model(
        NETWORKS_DIR, arguments['model'],
        device=arguments['device'],
        hidden_dim=arguments['hidden_dim'],
        input_dim=arguments['input_dim'],
        output_dim=arguments['output_dim'],
        is_maskable=arguments['disable_masking'],
        is_tracking_weights=arguments['track_weights'],
        is_rewindable=arguments['enable_rewinding'],
        is_growable=arguments['growing_rate'] > 0,
        outer_layer_pruning=arguments['outer_layer_pruning'],
        maintain_outer_mask_anyway=(
                                       not arguments['outer_layer_pruning']) and (
                                           "Structured" in arguments['prune_criterion']),
        l0=arguments['l0'],
        l0_reg=arguments['l0_reg'],
        N=arguments['N'],
        beta_ema=arguments['beta_ema'],
        l2_reg=arguments['l2_reg']
).to(arguments['device'])

10


In [7]:
load_checkpoint(arguments, model, out)

In [8]:
def load_checkpoint(path, model, out):
    with open(path, 'rb') as f:
        state = pickle.load(f)
    try:
        model.load_state_dict(state)
    except KeyError as e:
        print(list(state.keys()))
        raise e
    out(f"Loaded checkpoint {path}")

In [9]:
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-12_03.45.39_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-12_04.48.17_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=2345/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-15_19.21.19_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=3456/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-15_19.25.40_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=4567/models/Conv6_finished.pickle'
# path = '/nfs/students/ayle/guided-research/results/Conv6/2021-07-15_19.25.40_model=Conv6_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=4567/models/Conv6_finished.pickle'

# path = '/nfs/students/ayle/guided-research/results/LeNet5/2021-07-11_03.10.29_model=LeNet5_dataset=FASHION_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/LeNet5_finished.pickle'

# path = '/nfs/students/ayle/guided-research/results/ResNet18/2021-07-13_11.03.15_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

path = '/nfs/students/ayle/guided-research/results/ResNet18/2021-07-26_22.46.19_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'
# path2 = '/nfs/students/ayle/guided-research/results/ResNet18/2021-07-26_23.33.17_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=2345/models/ResNet18_finished.pickle'
# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-07-26_23.35.18_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=3456/models/ResNet18_finished.pickle'
# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-07-26_23.35.46_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=4567/models/ResNet18_finished.pickle'
# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-07-26_23.36.23_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=5678/models/ResNet18_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/VGG16/2021-08-22_11.02.10_model=VGG16_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/VGG16_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-08-29_13.57.22_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-08-29_18.19.04_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# path = '/nfs/homedirs/ayle/guided-research/SNIP-it/gitignored/results/ResNet18/2021-08-24_11.36.47_model=ResNet18_dataset=CIFAR10_prune-criterion=EarlyJohn_pruning-limit=0.94_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# path = '/nfs/students/ayle/guided-research/gitignored/results/tests/2021-09-21_10.19.16_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# model trained on augerino augmentations
# path = '/nfs/students/ayle/guided-research/gitignored/results/tests/2021-09-24_11.30.01_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle'

# augerino
# path = '/nfs/students/ayle/guided-research/gitignored/results/invariances/aug_no_trans_trained.pt'

load_checkpoint(path, model, out)


Loaded checkpoint /nfs/students/ayle/guided-research/results/ResNet18/2021-07-26_22.46.19_model=ResNet18_dataset=CIFAR10_prune-criterion=EmptyCrit_pruning-limit=0.0_prune-freq=1_prune-delay=0_outer-layer-pruning=1_prune-to=10_rewind-to=0_train-scheme=DefaultTrainer_seed=1234/models/ResNet18_finished.pickle


In [10]:
device = arguments['device']

In [11]:
# # augmentation stuff
# width = torch.load("/nfs/students/ayle/guided-research/gitignored/results/invariances/aug_no_trans_trained_width.pt")

# from augerino import models 
# aug = models.UniformAug()
# aug.set_width(width.data)

In [12]:
# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments,
    mean=arguments['mean'],
    std=arguments['std']
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments,
    mean=arguments['mean'],
    std=arguments['std']
)

Using mean (0.4914, 0.4822, 0.4465)
Files already downloaded and verified
Files already downloaded and verified


/nfs/homedirs/ayle/miniconda3/envs/inv/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Using mean (0.4914, 0.4822, 0.4465)
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [13]:
# load OOD data
ood_prune_loader, _ = find_right_model(
    DATASETS, arguments['ood_data_set_prune'],
    arguments=arguments,
    mean=arguments['mean'],
    std=arguments['std']
)

# get loss function
loss = find_right_model(
    LOSS_DIR, arguments['loss'],
    device=device,
    l1_reg=arguments['l1_reg'],
    lp_reg=arguments['lp_reg'],
    l0_reg=arguments['l0_reg'],
    hoyer_reg=arguments['hoyer_reg']
)

# get optimizer
optimizer = find_right_model(
    OPTIMS, arguments['optimizer'],
    params=model.parameters(),
    lr=arguments['learning_rate'],
    weight_decay=arguments['l2_reg'] if not arguments['l0'] else 0
)

Using mean (0.4914, 0.4822, 0.4465)
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [14]:
backup_model = deepcopy(model)

In [15]:
run_name = f'_model={arguments["model"]}_dataset={arguments["data_set"]}_ood-dataset={arguments["ood_data_set"]}' + \
           f'_attack={arguments["attack"]}_epsilon={arguments["epsilon"]}_prune-criterion={arguments["prune_criterion"]}' + \
           f'_pruning-limit={arguments["pruning_limit"]}_prune-freq={arguments["prune_freq"]}_prune-delay={arguments["prune_delay"]}' + \
           f'_rewind-to={arguments["rewind_to"]}_train-scheme={arguments["train_scheme"]}_seed={arguments["seed"]}'


criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        orig_scores=True,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments,
    )

# build trainer
trainer = find_right_model(
    TRAINERS_DIR, arguments['train_scheme'],
    model=model,
    loss=loss,
    optimizer=optimizer,
    device=device,
    arguments=arguments,
    train_loader=train_loader,
    test_loader=test_loader,
    ood_loader=ood_loader,
    ood_prune_loader=ood_prune_loader,
    metrics=metrics,
    criterion=criterion,
    run_name=run_name
)

trainer.train()

Made datestamp: 2021-09-29_18.48.15_model=ResNet18_dataset=CIFAR10_ood-dataset=SVHN_attack=FGSM_epsilon=6_prune-criterion=SNAP_pruning-limit=0.0_prune-freq=1_prune-delay=0_rewind-to=0_train-scheme=DefaultTrainer_seed=1234
Started training


In [16]:
orig_grads = criterion.grads_abs

In [17]:
orig_mean = criterion.scores_mean
orig_std = criterion.scores_std
layer_names = list(orig_grads.keys())

AttributeError: 'SNAP' object has no attribute 'scores_mean'

In [ ]:
# for name, val in orig_grads.items():
#     orig_grads[name] = (val - orig_mean[name]) / (1e-8 + orig_std[name])

In [ ]:
# backup_model = deepcopy(model)

In [18]:
arguments['batch_size'] = 2

# load data
train_loader, test_loader = find_right_model(
    DATASETS, arguments['data_set'],
    arguments=arguments,
    mean=arguments['mean'],
    std=arguments['std']
)

# load OOD data
_, ood_loader = find_right_model(
    DATASETS, arguments['ood_data_set'],
    arguments=arguments,
    mean=arguments['mean'],
    std=arguments['std']
)

Using mean (0.4914, 0.4822, 0.4465)
Files already downloaded and verified
Files already downloaded and verified
Using mean (0.4914, 0.4822, 0.4465)
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [62]:
kl = torch.nn.KLDivLoss()

In [ ]:
np.mean(norms)

In [66]:
norms = []
per_layer_norms = []
for i, (x, y) in enumerate(tqdm(test_loader)):
    if i == int(len(test_loader)*0.1): break
    
    model = deepcopy(backup_model)
    model.eval()

    x = x.cuda()

#     new_x = [x]
#     for im in x:
#         for _ in range(5):
#             image = aug(im.unsqueeze(0))
#             new_x.append(image)
#     x = torch.cat(new_x)

    out = model(x)
    preds = out.argmax(dim=-1, keepdim=True).flatten()

    train_loader = [(x, preds)]

#     train_loader = []
#     for im in x:
#         batch = []
        
#         batch.append(im.unsqueeze(0))
#         for _ in range(1):
#             new_x = aug(deepcopy(im.unsqueeze(0)))
#             batch.append(new_x)
#         batch = torch.cat(batch)
#         out = model(batch)
#         pred = out.argmax(dim=-1, keepdim=True).flatten()
        
#         train_loader.append((batch, pred))
        
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                        train_loader=train_loader,
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        if j == 0: continue #####################################################################
#         grad3 = (grad2 - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
        grad3 = grad2
#         layer_norms.append(torch.norm(grad1 - grad3, p=2).cpu().detach().numpy())
#         layer_norms.append(cosine(grad1.cpu().detach().numpy(), grad3.cpu().detach().numpy()))
        layer_norms.append(kl(torch.log(torch.nn.functional.softmax(grad1, -1)).unsqueeze(0), torch.nn.functional.softmax(grad3, -1).unsqueeze(0)).cpu().numpy())
#         layer_norms.append(MMD(grad1.unsqueeze(0), grad3.unsqueeze(0), kernel="rbf").cpu().detach().numpy())
    per_layer_norms.append(layer_norms)
    norms.append(np.mean(layer_norms))

 10%|█         | 500/5000 [00:55<08:15,  9.08it/s]


In [67]:
# Load OOD data
_, ood_loader = find_right_model(
    DATASETS, "SVHN",
    arguments=arguments,
    mean=arguments['mean'],
    std=arguments['std']
)

Using mean (0.4914, 0.4822, 0.4465)
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/train_32x32.mat
Using downloaded and verified file: /nfs/students/ayle/guided-research/gitignored/data/test_32x32.mat


In [68]:
# OOD data

ood_norms = []
ood_per_layer_norms = []

for i, (x, y) in enumerate(tqdm(ood_loader)):
    if i == int(len(test_loader)*0.1): break
    
    model = deepcopy(backup_model)
    model.eval()

    x = x.cuda()

#     new_x = [x]
#     for im in x:
#         for _ in range(5):
#             image = aug(im.unsqueeze(0))
#             new_x.append(image)
#     x = torch.cat(new_x)

    out = model(x)
    preds = out.argmax(dim=-1, keepdim=True).flatten()

    train_loader = [(x, preds)]

#     train_loader = []
#     for im in x:
#         batch = []
        
#         batch.append(im.unsqueeze(0))
#         for _ in range(1):
#             new_x = aug(deepcopy(im.unsqueeze(0)))
#             batch.append(new_x)
#         batch = torch.cat(batch)
#         out = model(batch)
#         pred = out.argmax(dim=-1, keepdim=True).flatten()
        
#         train_loader.append((batch, pred))
    
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                        train_loader=train_loader,
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        if j == 0: continue #####################################################################
#         grad3 = (grad2 - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
        grad3 = grad2
#         layer_norms.append(torch.norm(grad1 - grad3, p=2).cpu().detach().numpy())
#         layer_norms.append(cosine(grad1.cpu().detach().numpy(), grad3.cpu().detach().numpy()))
        layer_norms.append(kl(torch.log(torch.nn.functional.softmax(grad1, -1)).unsqueeze(0), torch.nn.functional.softmax(grad3, -1).unsqueeze(0)).cpu().numpy())
#         layer_norms.append(MMD(grad1.unsqueeze(0), grad3.unsqueeze(0), kernel="rbf").cpu().detach().numpy())
    per_layer_norms.append(layer_norms)
    ood_norms.append(np.mean(layer_norms))

  4%|▍         | 500/13016 [00:58<24:16,  8.59it/s] 


In [ ]:
# Attacks
from torchvision import datasets
from utils.constants import DATASET_PATH
mean=arguments['mean']
std=arguments['std']
test_set = datasets.CIFAR10(root=DATASET_PATH, train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=arguments['batch_size'],
        shuffle=False,
        pin_memory=True,
        num_workers=4
    )

trans = transforms.Normalize(mean, std)

# Attacks
ood_norms = []
attack_per_layer_norms = []

for i, (x, y) in enumerate(tqdm(test_loader)):
    if i == int(len(test_loader)*0.1): break
    
    model = deepcopy(backup_model)
    model.eval()
        
    adv_results, predictions = construct_adversarial_examples(x, y, 'FGSM', model, model.device, 8, False, False)
    _, advs, success = adv_results
    x = advs.cpu()
        
    new_x = []
    for image in x:
        image = trans(image.squeeze()).unsqueeze(0)
        new_x.append(image)
    x = torch.cat(new_x)
    
    x = x.cuda()
    
#     new_x = [x]
#     for im in x:
#         for _ in range(5):
#             image = aug(im.unsqueeze(0))
#             new_x.append(image)
#     x = torch.cat(new_x)
        
    model.eval()
    
    out = model(x.cuda())
    preds = out.argmax(dim=-1, keepdim=True).flatten()
    
    # get criterion
    criterion = find_right_model(
        CRITERION_DIR, arguments['prune_criterion'],
        model=model,
        limit=arguments['pruning_limit'],
        start=0.5,
        steps=arguments['snip_steps'],
        device=arguments['device'],
        arguments=arguments
    )
    
    criterion.prune(arguments['pruning_limit'],
                    train_loader=[(x, preds)],
                      ood_loader=None,
                      local=arguments['local_pruning'],
                      manager=None)
    
    layer_norms = []
    for j, (grad1, grad2) in enumerate(zip(orig_grads.values(), criterion.grads_abs.values())):
        grad3 = (grad2.cpu() - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
#         grad3 = grad2
        layer_norms.append(torch.norm(grad1 - grad3, p=5).cpu().detach().numpy())
    attack_per_layer_norms.append(layer_norms)
    ood_norms.append(np.mean(layer_norms))

In [ ]:
# DS
class DS(Dataset):

    def __init__(self, images, labels, mean, std):
        self.images = images
        self.labels = labels
        self.mean = [0.4914, 0.4822, 0.4465]
        self.std = [0.2471, 0.2435, 0.2616]
        self.transforms = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ]
        )

    def __getitem__(self, item):
        image = self.images[item] / 255
        image = self.transforms(image.transpose((1, 2, 0)))
        return image.to(torch.float32), torch.tensor(self.labels[item], dtype=torch.float32)

    def __len__(self):
        return len(self.images)

ds_path = os.path.join(DATASET_PATH, "cifar10_corrupted")
aurocs = []
ds_per_layer_norms = []
        
for ds_dataset_name in os.listdir(ds_path):
    if ds_dataset_name.endswith('5.npz'):
        npz_dataset = np.load(os.path.join(ds_path, ds_dataset_name))

        ds_dataset = CIFAR10C(npz_dataset["images"], npz_dataset["labels"], arguments["mean"], arguments["std"])
        ds_loader = torch.utils.data.DataLoader(
            ds_dataset,
            batch_size=arguments['batch_size'],
            shuffle=False,
            pin_memory=True,
            num_workers=4
        )

        ood_norms = []
        for i, (x, y) in enumerate(ds_loader):
            
            model = deepcopy(backup_model)
            model.eval()
            
            out = model(x.cuda())
            preds = out.argmax(dim=-1, keepdim=True).view_as(y)

            # get criterion
            criterion = find_right_model(
                CRITERION_DIR, arguments['prune_criterion'],
                model=model,
                limit=arguments['pruning_limit'],
                start=0.5,
                steps=arguments['snip_steps'],
                device=arguments['device'],
                arguments=arguments
            )

            criterion.prune(arguments['pruning_limit'],
                              train_loader=[(x, preds)],
                              ood_loader=None,
                              local=arguments['local_pruning'],
                              manager=None)

            layer_norms = []
            for grad1, grad2 in zip(orig_grads.values(), criterion.grads_abs.values()):
                grad3 = (grad2.cpu() - orig_mean[layer_names[j]]) /  (1e-8 + orig_std[layer_names[j]])
#                 grad3 = grad2
                layer_norms.append(torch.norm(grad1 - grad3, p=5).cpu().detach().numpy())
            ds_per_layer_norms.append(layer_norms)
            ood_norms.append(np.mean(layer_norms))

        auroc = calculate_auroc(np.concatenate((np.zeros_like(norms), np.ones_like(ood_norms))), np.concatenate((norms, ood_norms)))
        print('AUROC', auroc)
        aurocs.append(auroc)
        
print('Mean AUROC', np.mean(aurocs))

In [59]:
np.mean(ood_norms)

0.21531701

In [60]:
norms = np.array(norms)
ood_norms = np.array(ood_norms)

In [61]:
calculate_auroc(np.concatenate((np.zeros_like(norms), np.ones_like(ood_norms))), np.concatenate((norms, ood_norms)))

0.9230079999999999

In [ ]:
plt.plot(np.array(per_layer_norms).mean(0), label='In-distribution')
plt.plot(np.array(ood_per_layer_norms).mean(0), label='Out-Of-Distribution')
# plt.plot(np.array(attack_per_layer_norms).mean(0), label='Attack (FGSM \u03B5 = 8)')
# plt.plot(np.array(ds_per_layer_norms).mean(0), label='Distribution Shifts')
plt.legend()
plt.xlabel('ResNet18 layers')
plt.ylabel('L2 distance to original scores')

In [ ]:
# np.array(ood_per_layer_norms).mean(0)

In [ ]:
thresholds, _ = torch.topk(torch.tensor(norms), int(len(norms)*0.05), sorted=True)

In [ ]:
threshold = thresholds[-1]

In [ ]:
num_detected = 0
for ood_norm in ood_norms:
    if ood_norm >= threshold:
        num_detected += 1

In [ ]:
num_detected / len(ood_norms)

In [36]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def MMD(x, y, kernel):
    """Emprical maximum mean discrepancy. The lower the result
       the more evidence that distributions are the same.

    Args:
        x: first sample, distribution P
        y: second sample, distribution Q
        kernel: kernel type such as "multiscale" or "rbf"
    """
    xx, yy, zz = torch.mm(x, x.t()), torch.mm(y, y.t()), torch.mm(x, y.t())
    rx = (xx.diag().unsqueeze(0).expand_as(xx))
    ry = (yy.diag().unsqueeze(0).expand_as(yy))
    
    dxx = rx.t() + rx - 2. * xx # Used for A in (1)
    dyy = ry.t() + ry - 2. * yy # Used for B in (1)
    dxy = rx.t() + ry - 2. * zz # Used for C in (1)
    
    XX, YY, XY = (torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device),
                  torch.zeros(xx.shape).to(device))
    
    if kernel == "multiscale":
        
        bandwidth_range = [0.2, 0.5, 0.9, 1.3]
        for a in bandwidth_range:
            XX += a**2 * (a**2 + dxx)**-1
            YY += a**2 * (a**2 + dyy)**-1
            XY += a**2 * (a**2 + dxy)**-1
            
    if kernel == "rbf":
      
        bandwidth_range = [10, 15, 20, 50]
        for a in bandwidth_range:
            XX += torch.exp(-0.5*dxx/a)
            YY += torch.exp(-0.5*dyy/a)
            XY += torch.exp(-0.5*dxy/a)
      
      

    return torch.mean(XX + YY - 2. * XY)